### 1번 과제: 게시글 제목 다듬기

- 상황 : 대학교에서 교수님들과 대학생들에게 공개된 게시판을 운영하고 있는 운영자로써, 부적절한 제목의 게시글이 올라왔을 때 AI를 활용하여 감지 후, 부적절한 표현을 적절하게 수정한 뒤 게시글이 업로드되도록 하는 시스템을 개발하고자 함
- 제약사항
    1. 게시글이 올라오는 상황은 주어진 텍스트 파일을 한 줄씩 읽는 상황으로 가정
    2. 주어진 제목은 비속어가 있을 수도 있고 없을 수도 있음
    3. 게시글 내용은 고려하지 않음
- 코드 구현 시 참고사항
    1. 제목을 입력으로 받는 함수로 구현할 것
    2. 서비스에서 활용할 때 비속어가 있는지 없는지 확인이 가능해야함
    3. 비속어가 있는 경우 변환된 제목을 같이 받아와야함
- 프롬프트 구성 방식
    1. system: 배경상황 혹은 페르소나 설정 
    2. instruction: 사용자의 입력이 어떻게 들어오는지 명시, 비속어가 있는 경우, 없는 경우 처리 지시
    3. context
    4. input data: 입력값 (제목)
    4. output specification: 출력값의 형태를 지정

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
# OPEN AI API 키 설정
load_dotenv()

True

In [2]:
import os

MY_API_KEY = os.environ['OPENAI_API_KEY']

In [3]:
client = OpenAI(api_key=MY_API_KEY)

In [ ]:
title = ""
system_prompt = """너는 대학교 게시판을 관리하는 관리자야.
다음의 제목을 보고, 비속어가 포함되어있는지와 포함되어있으면, 비속어를 적절하게 수정해줘"""

question = f"""
제목: {title}
"""
output_format = """
출력 형식은 마크다운을 제외한 JSON포맷으로
bad_word: 예, 아니오
fixed_title: 수정된 제목
"""

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': question + output_format},
    ],
    max_tokens=150,
    temperature=0
)

print(response.choices[0].message.content)


{
  "bad_word": "아니오",
  "fixed_title": ""
}


In [18]:
import json

data = response.choices[0].message.content
json.loads(data)

{'bad_word': '아니오', 'fixed_title': ''}

In [4]:
import json

def check_title(title:str) -> dict:
    system_prompt = """
    너는 대학교 게시판을 관리하는 관리자야.
    다음의 제목을 보고, 비속어가 포함되어있는지와 포함되어있으면, 
    비속어를 적절하게 수정해줘"""

    question = f"""
    제목: {title}
    """
    output_format = """
    출력 형식은 마크다운을 제외한 JSON포맷으로
    bad_word: 예, 아니오
    fixed_title: 수정된 제목
    """
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': question + output_format},
        ],
        max_tokens=150,
        temperature=0
    )

    return json.loads(response.choices[0].message.content)

In [5]:
with open('title.txt', 'r', encoding='utf-8') as f:
    titles = f.readlines()

for title in titles:
    print('원래 제목: ', title)
    result = check_title(title)

    print('비속어 포함 여부: ', result['bad_word'])
    if result['bad_word'] == '예':
        print('수정된 제목: ', result['fixed_title'])
    print('-'*100)


원래 제목:  이딴 거 왜 배워야 돼, 씨발

비속어 포함 여부:  예
수정된 제목:  이런 거 왜 배워야 돼
----------------------------------------------------------------------------------------------------
원래 제목:  개같은 과제 또 내냐

비속어 포함 여부:  예
수정된 제목:  너무 어려운 과제 또 내냐
----------------------------------------------------------------------------------------------------
원래 제목:  수업 내용에 대해 질문이 있습니다

비속어 포함 여부:  아니오
----------------------------------------------------------------------------------------------------
원래 제목:  이런 병신 같은 규칙 좀 바꿔라

비속어 포함 여부:  예
수정된 제목:  이런 부적절한 규칙 좀 바꿔라
----------------------------------------------------------------------------------------------------
원래 제목:  과제 제출 기한 문의드립니다

비속어 포함 여부:  아니오
----------------------------------------------------------------------------------------------------
원래 제목:  게시판 이용 규칙 안내 부탁드립니다

비속어 포함 여부:  아니오
----------------------------------------------------------------------------------------------------
원래 제목:  강의 자료 추가 업로드 요청합니다

비속어 포함 여부:  아니오
--------------------------------

### 2번 과제: 영어 단어장 만들기

- 상황 : 영어 공부를 위해 영어 단어들을 제시하는 AI 서비스를 개발하려고 합니다. 단어장의 구성은 영단어, 한국어 의미, 유사어, 반의어, 예문 등을 포함해야 합니다.
- 제약사항
    1. 제안받는 단어의 난이도를 설정할 수 있어야합니다.
    2. 한번 제안받은 단어는 다시 제안받지 않도록 해야합니다.
- 코드 구현 시 참고사항
    1. 사용자의 입력에 따라 다음 단어를 제안받거나, 멈출 수 있어야합니다.
    2. 지금까지 제안받은 단어 목록을 지속적으로 관리할 수 있도록 클래스화, 파일관리를 활용합니다.

In [ ]:
level=""
word=""
mean=""
similar=""
antonym=""
example=""

# 난이도를 포함한 prompt 작성
system_prompt = """ 
1. 제안 받는 영단어의 난이도를 설정할 수 있어야 한다.
2. 한번 받은 영단어는 다시 제안 받지 않도록 해야 한다.
"""
# 출력 포맷 정해서 prompt에 적용
result = f""" 
난이도: {level}
"""
output_format = """ 
출력 형식은 마크다운을 제외한 JSON포맷으로
난이도: 상, 중, 하
단어: {word}, 뜻: {mean}
유사어: {similar}, 반의어: {antonym}
예문: {example}
"""
# 추천 받는 로직
response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': result + output_format},
    ],
    max_tokens=150,
    temperature=0
)
# 추천받은 영어단어를 prompt에 다시 넣을 수 있게 처리

# 파일관리 -> 이름, 난이도로 파일 저장 / 내용은 추천받은 영어단어들
# 클래스화 -> 이름, 난이도 관리

print(response.choices[0].message.content)


{
  "난이도": "중",
  "단어": "abundant",
  "뜻": "풍부한, 많은",
  "유사어": ["plentiful", "ample", "copious"],
  "반의어": ["scarce", "rare", "insufficient"],
  "예문": "The garden was abundant with colorful flowers during the spring."
}


In [24]:
import json

class EnglishWordAI:

    def __init__(self, name: str, level: str):
        """
        Args:
            name: 서비스를 사용하는 사람의 이름
            level: 단어 난이도 (예: easy, medium, hard)
        """
        self.name = name
        self.level = level
        self.filename = f"{self.name}_wordbook.json"
        self.words = self.load_words()
        
    def load_words(self):
        """ 기존 학습된 단어 파일 불러오기 """
        if os.path.exists(self.filename):
            with open(self.filename, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []
    
    def save_words(self):
        """ 지금까지 제안받은 단어 목록 저장 """
        with open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(self.words, f, ensure_ascii=False, indent=2)
    
    def get_word(self):
        """ 새로운 단어 추천 요청 및 중복 방지 """
        used_words = [w['word'] for w in self.words]

        system_prompt = """
        너는 영어 단어를 추천하는 AI야. 
        사용자의 영어 수준에 맞춰 단어 하나를 추천하고,
        해당 단어의 의미, 유의어, 반의어, 예문을 제공해줘. 
        이미 추천된 단어는 다시 추천하지 마.
        """

        user_prompt = f"""
        난이도: {self.level}
        이미 추천된 단어 목록: {', '.join(used_words) if used_words else '없음'}

        JSON 형식으로 출력해:
        {{
        "word": "추천된 단어",
        "meaning": "뜻",
        "similar": "유의어",
        "antonym": "반의어",
        "example": "예문"
        }}
        """

        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        return json.loads(response.choices[0].message.content)

ai = EnglishWordAI('jason', 'easy')

while True:
    if input('continue? (y/n): ') == 'n':
        break
    else:
        word_dict = ai.get_word()
        print(f"단어: {word_dict['word']}, 뜻: {word_dict['meaning']}")
        print(f"유사어: {word_dict['similar']}, 반의어: {word_dict['antonym']}")
        print(f"예문: {word_dict['example']}")

단어: happy, 뜻: feeling or showing pleasure or contentment
유사어: joyful, cheerful, content, 반의어: sad, unhappy, miserable
예문: She felt happy when she received the good news.


### 3번 과제: 면접 질문 생성하기

- 상황 : 서류에 합격한 뒤, 면접을 준비하기 위해 예상 면접 질문을 AI를 활용해 제안받으려고 합니다.  
면접 질문은 채용 공고와 작성한 자기소개서를 기반으로 추천받을 수 있어야하며, 예상 질문 뒤에 본인의 답변이 적절했는지 AI를 활용해 검토할 수 있어야 합니다.

In [ ]:
def interview() -> dict:
    with open('myprofile.txt', 'r', encoding='utf-8') as f:
        answer = f.read()

    # 면접 질문 생성기 AI - 채용공고, 자기소개서 -> 질문
    system_prompt = """ 
    채용공고와 자기소개서를 비교해보고 인성면접에 대한 예상 질문을 생성해줘.
    그리고 생성된 답변을 보고 평가 및 피드백을 해줘.
    """
    # 면접 답변 평가기 AI - 질문, 답변 -> 평가 및 피드백
    output_format = """ 
    출력 형식은 마크다운을 제외한 JSON포맷으로
    {{
    "question": "생성된 예상 질문",
    "answer": "내 자소서 내용",
    "feedback": "답변에 대한 평가 및 피드백"
    }}
    """

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': output_format},
        ],
        max_tokens=300, 
        temperature=0
    )
    return json.loads(response.choices[0].message.content)

# 실행
if __name__ == "__main__":
    result = interview()
    print("예상 질문:", result.get("question"))
    print("내 답변:", result.get("answer"))
    print("피드백:", result.get("feedback"))

예상 질문: 우리 회사의 팀워크와 협업을 중요시하는 문화에 대해 어떻게 생각하시며, 과거 경험 중 팀 내 갈등을 해결한 사례가 있다면 말씀해 주세요.
내 답변: 저는 대학 시절 프로젝트 팀에서 팀원 간 의견 차이로 갈등이 발생했을 때, 중재자의 역할을 맡아 각자의 의견을 경청하고 공통점을 찾아 조율하였습니다. 이를 통해 팀원들이 서로 이해하고 협력할 수 있었으며, 프로젝트를 성공적으로 마무리할 수 있었습니다. 이러한 경험은 협업의 중요성을 깨닫게 해주었고, 앞으로도 팀워크를 중시하는 환경에서 적극적으로 소통하며 기여하고자 합니다.
피드백: 답변이 구체적인 경험을 바탕으로 팀워크와 갈등 해결 능력을 잘 보여주고 있습니다. 다만, 갈등 상황에서 본인이 구체적으로 어떤 행동을 했는지 더 명확히 설명하면 좋겠습니다. 예를 들어, 어떤 방법으로 의견을 조율했는지, 어떤 커뮤니케이션 스킬을 사용했는지 구체적으로 언급하면 면접관에게 더 신뢰감을 줄 수 있습니다.


## 실전 1. 게시판 관리 자동화

- 상황 : 하루에 1000건 정도 게시글이 올라오는 취업준비생 카페를 관리하는 운영자로써 작성되는 게시물의 종류를 구분하고, 작성되는 게시물에 따라 적절하게 반응하는 자동화 AI 서비스를 개발하고자 합니다.

- 제약사항
    1. 게시물은 제목없이 250자의 내용만으로 구성되어있습니다.
    2. 게시물의 종류는 홍보 글, 고민 상담 글, 신규 채용공고 글, 합격 후기 글 등이 있습니다.

- 응용분야 : 고객 문의 관리 자동화

In [30]:
contents=""
category=""

system_prompt = """ 
- 상황 : 하루에 1000건 정도 게시글이 올라오는 취업준비생 카페를 관리하는 운영자로써 작성되는 게시물의 종류를 구분하고, 작성되는 게시물에 따라 적절하게 반응하는 자동화 AI 서비스를 개발하고자 합니다.

- 제약사항
    1. 게시물은 제목없이 250자의 내용만으로 구성되어있습니다.
    2. 게시물의 종류는 홍보 글, 고민 상담 글, 신규 채용공고 글, 합격 후기 글 등이 있습니다.

- 응용분야 : 고객 문의 관리 자동화
"""

output_format = """ 
출력 형식은 마크다운을 제외한 JSON포맷으로
게시물: {contents}
종류: {category}
"""

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': output_format},
    ],
    max_tokens=150,
    temperature=0
)

print(response.choices[0].message.content)

게시물: "이번에 새로 오픈한 학원에서 취업 준비를 위한 특별 강의를 진행합니다. 관심 있으신 분들은 연락 주세요!"
종류: "홍보 글"


## 실전 2. 주가 정보 수집 스크래퍼 제작

- 상황 : [업종별 시세](https://finance.naver.com/sise/sise_group.naver?type=upjong)에서 업종별로 전일대비 얼마나 상승, 하락했는지를 특정 시간에 받아볼 수 있도록 자동화된 시스템을 개발하려고 합니다.

In [ ]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://finance.naver.com/sise/sise_group.naver?type=upjong')
soup = BeautifulSoup(res.text, 'html.parser')


stocks = []
rows = soup.select('table.type_1 tr')[2:] 

for row in rows:
    cols = row.select('td')
    if len(cols) < 2:  
        continue

    name_tag = cols[0].select_one('a')
    change_tag = cols[1].select_one('span')

    if name_tag and change_tag:
        stock = {
            '업종': name_tag.text.strip(),
            '전일대비': change_tag.text.strip()
        }
        stocks.append(stock)

for stock in stocks:
    print(stock)

prompt = f"""
다음은 업종별 시세 정보입니다. 
각각의 업종이 전일 대비 상승했는지 하락했는지를 특정 시간에 받아볼 수 있도록 해주세요.
숫자는 참고용이며, 정성적 요약 위주로 얼마나 변화했는지 간단히 정리해 주세요.

{stocks}
"""
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

print(response.choices[0].message.content)


{'업종': '담배', '전일대비': '+5.42%'}
{'업종': '레저용장비와제품', '전일대비': '+3.68%'}
{'업종': '기계', '전일대비': '+3.40%'}
{'업종': '건축자재', '전일대비': '+3.37%'}
{'업종': '에너지장비및서비스', '전일대비': '+3.36%'}
{'업종': '전기유틸리티', '전일대비': '+3.29%'}
{'업종': '판매업체', '전일대비': '+3.15%'}
{'업종': '방송과엔터테인먼트', '전일대비': '+3.10%'}
{'업종': '생물공학', '전일대비': '+2.78%'}
{'업종': '창업투자', '전일대비': '+2.43%'}
{'업종': '전기장비', '전일대비': '+2.41%'}
{'업종': '전자장비와기기', '전일대비': '+2.40%'}
{'업종': '생명과학도구및서비스', '전일대비': '+2.28%'}
{'업종': '가구', '전일대비': '+2.09%'}
{'업종': '양방향미디어와서비스', '전일대비': '+1.94%'}
{'업종': '호텔,레스토랑,레저', '전일대비': '+1.93%'}
{'업종': '인터넷과카탈로그소매', '전일대비': '+1.85%'}
{'업종': '건강관리기술', '전일대비': '+1.72%'}
{'업종': '백화점과일반상점', '전일대비': '+1.67%'}
{'업종': '증권', '전일대비': '+1.65%'}
{'업종': '출판', '전일대비': '+1.63%'}
{'업종': '음료', '전일대비': '+1.61%'}
{'업종': '은행', '전일대비': '+1.48%'}
{'업종': '무선통신서비스', '전일대비': '+1.46%'}
{'업종': '제약', '전일대비': '+1.45%'}
{'업종': '포장재', '전일대비': '+1.42%'}
{'업종': '도로와철도운송', '전일대비': '+1.42%'}
{'업종': '건강관리업체및서비스', '전일대비': '+1.38%'}
{'업종': '식품과기본식료품소매', '전일대비': '+1